# Need to check
1.Log transformation on response(yes) <br>
2.Log transformation on cont(no)<br>
3.Feature engineering and Feature selection<br>
* Categorize Cont
* Remove some Cat

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

from sklearn.cross_validation import train_test_split

In [2]:
train = pd.read_csv("../Data/train.csv")
test = pd.read_csv("../Data/test.csv")

In [3]:
del train["id"]
del test["id"]

# Feature engineering

Feature selection on cat

remove all variables with ratio over 0.98

In [4]:
catFeature = []
for i in range(1,116):
    catFeature.append("cat%i" % i)
    
ratio = []

for col in catFeature:
    max_ratio = train[col].value_counts().max()/len(train[col])
    ratio.append(max_ratio)

In [5]:
rmFeature = np.array(catFeature)[np.array(ratio)>=1]

In [6]:
for i in rmFeature:
    del train[i]
    del test[i]    

Modify variables with many categories

In [7]:
train["cat109"] = train["cat109"].apply(lambda x: "other" if x not in ("BI","AB") else x)

cat_110_domi = train.cat110.value_counts().index[(train.cat110.value_counts()/len(train.cat109))>0.01].values
train["cat110"] = train["cat110"].apply(lambda x: "other" if x not in cat_110_domi else x)

cat_113_domi = train.cat113.value_counts().index[(train.cat113.value_counts()/len(train.cat113))>0.005].values
train["cat113"] = train["cat113"].apply(lambda x: "other" if x not in cat_113_domi else x)

Take log of y

In [8]:
# train["loss"] = log(train["loss"])

In [9]:
X = train.ix[:,:-1]
y = train.ix[:,-1]

Taking log on the insurance data will not give much influence on the error, from the eda, we see that the continuous variables do not have linear relation with the response, we need to categorize them.

In [10]:
breakpoint_normal = [-0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
breakpoint2 = [-0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1.0]
breakpoint4 = [-0.01,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
breakpoint5 = [-0.01,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
breakpoint8 = [-0.01,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
breakpoint14 = [-0.01,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1.0]
breakpoint_special = [breakpoint2,breakpoint4,breakpoint5,breakpoint8,breakpoint14]

In [11]:
columns_normal_cut = ['cont1', 'cont3', 'cont6', 'cont7', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13',]
columns_special_cut = ['cont2','cont4','cont5','cont8','cont14']

In [12]:
# normal cut
for i in columns_normal_cut:
    X[i] = pd.cut(X[i],breakpoint_normal)

# special cut
k = 0
for i in columns_special_cut:
    X[i] = pd.cut(X[i],breakpoint_special[k])

In [13]:
# transform the cats to dummy
obj = X.dtypes.index[X.dtypes=="category"]
for i in obj:
    new_cat = pd.get_dummies(X[i],prefix=i,drop_first=True)
    del X[i]
    X = pd.concat([X,new_cat],axis = 1)

In [14]:
# transform the cats to dummy
obj = X.dtypes.index[X.dtypes=="object"]
for i in obj:
    new_cat = pd.get_dummies(X[i],prefix=i,drop_first=True)
    del X[i]
    X = pd.concat([X,new_cat],axis = 1)

In [15]:
# Train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [16]:
stop

NameError: name 'stop' is not defined

# Model

In [ ]:
# build random forest
from sklearn.ensemble import RandomForestRegressor

rft = RandomForestRegressor(n_estimators=350, max_features="sqrt", criterion='mse',
                            n_jobs=-1, random_state=42, verbose=3)

In [ ]:
rft.fit(X_train,y_train)
rft_pred = rft.predict(X_test)
err = abs(rft_pred-y_test).mean()

In [ ]:
# err feature selection

In [ ]:
# err 350 sqrt the best right now 1226.0620695749944

In [ ]:
# err 350 log2 1269.750625661697

In [ ]:
# err 300 sqrt 1226.7914547125808

In [ ]:
# err max_feature = log2

In [ ]:
# err max_feature = sqrt

In [ ]:
# err max_depth=8

In [ ]:
# err no constrain

# With the best forest structure, I will check the effect of different feature engineerings and try to achieve a good result

1.take log of the response

In [17]:
# build random forest
from sklearn.ensemble import RandomForestRegressor

rft = RandomForestRegressor(n_estimators=350, max_features="sqrt", criterion='mse',
                            n_jobs=-1, random_state=42, verbose=3)

In [18]:
y_log_train = np.log(y_train)

In [19]:
# fit model
rft.fit(X_train,y_log_train)
rft_pred = rft.predict(X_test)
err = abs(np.exp(rft_pred)-y_test).mean()

building tree 1 of 350
building tree 2 of 350
building tree 3 of 350
building tree 4 of 350
building tree 5 of 350
building tree 6 of 350
building tree 7 of 350
building tree 8 of 350
building tree 9 of 350
building tree 10 of 350
building tree 11 of 350
building tree 12 of 350
building tree 13 of 350
building tree 14 of 350
building tree 15 of 350
building tree 16 of 350
building tree 17 of 350
building tree 18 of 350
building tree 19 of 350
building tree 20 of 350
building tree 21 of 350
building tree 22 of 350
building tree 23 of 350
building tree 24 of 350


[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.3s


building tree 25 of 350
building tree 26 of 350
building tree 27 of 350
building tree 28 of 350
building tree 29 of 350
building tree 30 of 350
building tree 31 of 350
building tree 32 of 350
building tree 33 of 350
building tree 34 of 350
building tree 35 of 350
building tree 36 of 350
building tree 37 of 350
building tree 38 of 350
building tree 39 of 350
building tree 40 of 350
building tree 41 of 350
building tree 42 of 350
building tree 43 of 350
building tree 44 of 350
building tree 45 of 350
building tree 46 of 350
building tree 47 of 350
building tree 48 of 350
building tree 49 of 350
building tree 50 of 350
building tree 51 of 350
building tree 52 of 350
building tree 53 of 350
building tree 54 of 350
building tree 55 of 350
building tree 56 of 350
building tree 57 of 350
building tree 58 of 350
building tree 59 of 350
building tree 60 of 350
building tree 61 of 350
building tree 62 of 350
building tree 63 of 350
building tree 64 of 350
building tree 65 of 350
building tree 66

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   46.8s


building tree 122 of 350
building tree 123 of 350
building tree 124 of 350
building tree 125 of 350
building tree 126 of 350
building tree 127 of 350
building tree 128 of 350
building tree 129 of 350
building tree 130 of 350
building tree 131 of 350
building tree 132 of 350
building tree 133 of 350
building tree 134 of 350
building tree 135 of 350
building tree 136 of 350
building tree 137 of 350
building tree 138 of 350
building tree 139 of 350
building tree 140 of 350
building tree 141 of 350
building tree 142 of 350
building tree 143 of 350
building tree 144 of 350
building tree 145 of 350
building tree 146 of 350
building tree 147 of 350
building tree 148 of 350
building tree 149 of 350
building tree 150 of 350
building tree 151 of 350
building tree 152 of 350
building tree 153 of 350
building tree 154 of 350
building tree 155 of 350
building tree 156 of 350
building tree 157 of 350
building tree 158 of 350
building tree 159 of 350
building tree 160 of 350
building tree 161 of 350


[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  1.8min


building tree 281 of 350
building tree 282 of 350
building tree 283 of 350
building tree 284 of 350
building tree 285 of 350
building tree 286 of 350
building tree 287 of 350
building tree 288 of 350
building tree 289 of 350
building tree 290 of 350
building tree 291 of 350
building tree 292 of 350
building tree 293 of 350
building tree 294 of 350
building tree 295 of 350
building tree 296 of 350
building tree 297 of 350
building tree 298 of 350
building tree 299 of 350
building tree 300 of 350
building tree 301 of 350
building tree 302 of 350
building tree 303 of 350
building tree 304 of 350
building tree 305 of 350
building tree 306 of 350
building tree 307 of 350
building tree 308 of 350
building tree 309 of 350
building tree 310 of 350
building tree 311 of 350
building tree 312 of 350
building tree 313 of 350
building tree 314 of 350
building tree 315 of 350
building tree 316 of 350
building tree 317 of 350
building tree 318 of 350
building tree 319 of 350
building tree 320 of 350


[Parallel(n_jobs=-1)]: Done 350 out of 350 | elapsed:  2.4min finished
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 272 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 350 out of 350 | elapsed:    2.7s finished


In [20]:
err

1215.5031190067557